# imports/setup

In [1]:

import pandas as pd
from typing import List, Dict, Any
from datamodel import Listing, Trade

# backtester

In [2]:
class Backtester:
    def __init__(self, trader, listings: Dict[str, Listing], position_limit: Dict[str, int],
                 market_data: pd.DataFrame, trade_history: pd.DataFrame, file_name: str = None):
        self.trader = trader
        self.listings = listings
        self.market_data = market_data
        self.position_limit = position_limit
        self.fair_marks = {}
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.file_name = file_name

        self.observations = [Observation({}, {}) for _ in range(len(market_data))]

        self.current_position = {product: 0 for product in self.listings.keys()}
        self.pnl_history = []
        self.pnl = {product: 0 for product in self.listings.keys()}
        self.cash = {product: 0 for product in self.listings.keys()}
        self.trades = []
        self.sandbox_logs = []
        
    def run(self):
        traderData = ""
        
        timestamp_group_md = self.market_data.groupby('timestamp')
        timestamp_group_th = self.trade_history.groupby('timestamp')
        
        own_trades = defaultdict(list)
        market_trades = defaultdict(list)
        pnl_product = defaultdict(float)
        
        trade_history_dict = {}
        
        for timestamp, group in timestamp_group_th:
            trades = []
            for _, row in group.iterrows():
                symbol = row['symbol']
                price = row['price']
                quantity = row['quantity']
                buyer = row['buyer'] if pd.notnull(row['buyer']) else ""
                seller = row['seller'] if pd.notnull(row['seller']) else ""

                
                trade = Trade(symbol, int(price), int(quantity), buyer, seller, timestamp)
                
                trades.append(trade)
            trade_history_dict[timestamp] = trades
        
        
        for timestamp, group in timestamp_group_md:
            order_depths = self._construct_order_depths(group)
            order_depths_matching = self._construct_order_depths(group)
            order_depths_pnl = self._construct_order_depths(group)
            state = self._construct_trading_state(traderData, timestamp, self.listings, order_depths, 
                                 dict(own_trades), dict(market_trades), self.current_position, self.observations)
            orders, conversions, traderData = self.trader.run(state)
            products = group['product'].tolist()
            sandboxLog = ""
            trades_at_timestamp = trade_history_dict.get(timestamp, [])

            for product in products:
                new_trades = []
                for order in orders.get(product, []):
                    trades_done, sandboxLog = self._execute_order(timestamp, order, order_depths_matching, self.current_position, self.cash, trade_history_dict, sandboxLog)
                    new_trades.extend(trades_done)
                if len(new_trades) > 0:
                    own_trades[product] = new_trades
            self.sandbox_logs.append({"sandboxLog": sandboxLog, "lambdaLog": "", "timestamp": timestamp})

            trades_at_timestamp = trade_history_dict.get(timestamp, [])
            if trades_at_timestamp:
                for trade in trades_at_timestamp:
                    product = trade.symbol
                    market_trades[product].append(trade)
            else: 
                for product in products:
                    market_trades[product] = []

            
            for product in products:
                self._mark_pnl(self.cash, self.current_position, order_depths_pnl, self.pnl, product)
                self.pnl_history.append(self.pnl[product])
            self._add_trades(own_trades, market_trades)
        return self._log_trades(self.file_name)
    
    
    def _log_trades(self, filename: str = None):
        if filename is None:
            return 

        self.market_data['profit_and_loss'] = self.pnl_history

        output = ""
        output += "Sandbox logs:\n"
        for i in self.sandbox_logs:
            output += json.dumps(i, indent=2) + "\n"

        output += "\n\n\n\nActivities log:\n"
        market_data_csv = self.market_data.to_csv(index=False, sep=";")
        market_data_csv = market_data_csv.replace("\r\n", "\n")
        output += market_data_csv

        output += "\n\n\n\nTrade History:\n"
        output += json.dumps(self.trades, indent=2)

        with open(filename, 'w') as file:
            file.write(output)

            
    def _add_trades(self, own_trades: Dict[str, List[Trade]], market_trades: Dict[str, List[Trade]]):
        products = set(own_trades.keys()) | set(market_trades.keys())
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in own_trades.get(product, [])])
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in market_trades.get(product, [])])

    def _trade_to_dict(self, trade: Trade) -> dict[str, Any]:
        return {
            "timestamp": trade.timestamp,
            "buyer": trade.buyer,
            "seller": trade.seller,
            "symbol": trade.symbol,
            "currency": "SEASHELLS",
            "price": trade.price,
            "quantity": trade.quantity,
        }
        
    def _construct_trading_state(self, traderData, timestamp, listings, order_depths, 
                                 own_trades, market_trades, position, observations):
        state = TradingState(traderData, timestamp, listings, order_depths, 
                             own_trades, market_trades, position, observations)
        return state
    
        
    def _construct_order_depths(self, group):
        order_depths = {}
        for idx, row in group.iterrows():
            product = row['product']
            order_depth = OrderDepth()
            for i in range(1, 4):
                if f'bid_price_{i}' in row and f'bid_volume_{i}' in row:
                    bid_price = row[f'bid_price_{i}']
                    bid_volume = row[f'bid_volume_{i}']
                    if not pd.isna(bid_price) and not pd.isna(bid_volume):
                        order_depth.buy_orders[int(bid_price)] = int(bid_volume)
                if f'ask_price_{i}' in row and f'ask_volume_{i}' in row:
                    ask_price = row[f'ask_price_{i}']
                    ask_volume = row[f'ask_volume_{i}']
                    if not pd.isna(ask_price) and not pd.isna(ask_volume):
                        order_depth.sell_orders[int(ask_price)] = -int(ask_volume)
            order_depths[product] = order_depth
        return order_depths
    
        
        
    def _execute_buy_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]

        for price, volume in list(order_depth.sell_orders.items()):
            if price > order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(trade_volume + position[order.symbol]) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp))
                position[order.symbol] += trade_volume
                self.cash[order.symbol] -= price * trade_volume
                order_depth.sell_orders[price] += trade_volume
                order.quantity -= trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"
            

            if order_depth.sell_orders[price] == 0:
                del order_depth.sell_orders[price]
        
        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price < order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "SUBMISSION", "", timestamp))
                    order.quantity -= trade_volume
                    position[order.symbol] += trade_volume
                    self.cash[order.symbol] -= order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp

        return trades, sandboxLog
        
        
        
    def _execute_sell_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]
        
        for price, volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if price < order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(position[order.symbol] - trade_volume) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "", "SUBMISSION", timestamp))
                position[order.symbol] -= trade_volume
                self.cash[order.symbol] += price * abs(trade_volume)
                order_depth.buy_orders[price] -= abs(trade_volume)
                order.quantity += trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"

            if order_depth.buy_orders[price] == 0:
                del order_depth.buy_orders[price]

        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price > order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "", "SUBMISSION", timestamp))
                    order.quantity += trade_volume
                    position[order.symbol] -= trade_volume
                    self.cash[order.symbol] += order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp
                
        return trades, sandboxLog
        
        
        
    def _execute_order(self, timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog):
        if order.quantity == 0:
            return [], sandboxLog
        
        order_depth = order_depths[order.symbol]
        if order.quantity > 0:
            return self._execute_buy_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
        else:
            return self._execute_sell_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
    
    def _mark_pnl(self, cash, position, order_depths, pnl, product):
        order_depth = order_depths[product]
        
        best_ask = min(order_depth.sell_orders.keys())
        best_bid = max(order_depth.buy_orders.keys())
        mid = (best_ask + best_bid)/2
        fair = mid
        if product in self.fair_marks:
            get_fair = self.fair_marks[product]
            fair = get_fair(order_depth)
        
        pnl[product] = cash[product] + fair * position[product]

# trader

# backtest run

In [4]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

### setup

In [5]:
listings = {
    'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
    'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
    'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
    'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),
}

position_limit = {
    'PICNIC_BASKET2': 60,
    'CROISSANTS': 250,
    'JAMS': 350,
    'DJEMBES': 60,
}

### RUN FROM DATABOTTLE

In [6]:
products_to_include = ["CROISSANTS", "JAMS", "DJEMBES", "PICNIC_BASKET2"]

day = -1
market_data = pd.read_csv(f"./data/prices_round_2_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./data/trades_round_2_day_{day}.csv", sep=";", header=0)

market_data = market_data[market_data["product"].isin(products_to_include)].copy()
trade_history = trade_history[trade_history["symbol"].isin(products_to_include)].copy()

trader = Trader()
backtester = Backtester(trader, listings, position_limit, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)

{'CROISSANTS': -9868.5, 'JAMS': -1860.0, 'DJEMBES': 55.0, 'PICNIC_BASKET1': 35896.0}


In [7]:
for i in range(-1, 2):
    products_to_include = ["CROISSANTS", "JAMS", "DJEMBES", "PICNIC_BASKET2"]
    day = i
    market_data = pd.read_csv(f"./data/prices_round_2_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./data/trades_round_2_day_{day}.csv", sep=";", header=0)

    market_data = market_data[market_data["product"].isin(products_to_include)].copy()
    trade_history = trade_history[trade_history["symbol"].isin(products_to_include)].copy()

    trader = Trader()
    backtester = Backtester(trader, listings, position_limit, market_data, trade_history, f"clean_logs/trade_history_day_{i}.log")
    backtester.run()
    print(backtester.pnl)

{'CROISSANTS': -9868.5, 'JAMS': -1860.0, 'DJEMBES': 55.0, 'PICNIC_BASKET1': 35896.0}
{'CROISSANTS': 2304.0, 'JAMS': -3510.0, 'DJEMBES': -614.0, 'PICNIC_BASKET1': 12667.0}
{'CROISSANTS': -3612.0, 'JAMS': -6009.0, 'DJEMBES': 4272.0, 'PICNIC_BASKET1': 14042.0}


# backtest gridsearch

In [8]:


def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [9]:
import os


def run_backtests(trader, listings, position_limit, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [10]:
listings = {
    'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
    'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
    'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
    'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),
}

position_limit = {
    'PICNIC_BASKET2': 60,
    'CROISSANTS': 250,
    'JAMS': 350,
    'DJEMBES': 60,
}

In [12]:
products_to_include = ["CROISSANTS", "JAMS", "DJEMBES", "PICNIC_BASKET2"]

day = -1
market_data = pd.read_csv(f"./data/prices_round_2_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./data/trades_round_2_day_{day}.csv", sep=";", header=0)

market_data = market_data[market_data["product"].isin(products_to_include)].copy()
trade_history = trade_history[trade_history["symbol"].isin(products_to_include)].copy()

### run

In [3]:
from datamodel import OrderDepth, TradingState, Order  # (Assumes these are provided elsewhere)
from typing import List, Dict, Any


class Product:
    PICNIC_BASKET1 = "PICNIC_BASKET2"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    SYNTHETIC = "SYNTHETIC" # proxy basket
    SPREAD = "SPREAD" # basket - synthetic


PARAMS = {
    Product.SPREAD: {
        "default_spread_mean": 43.922489021349016,
        "default_spread_std": 82.78475819993702,
        "spread_sma_window": 1000,
        "spread_std_window": 40,
        "zscore_threshold": 7,
        "target_position": 58,
    },
}


BASKET_WEIGHTS = {
    Product.CROISSANTS: 6,
    Product.JAMS: 3,
    Product.DJEMBES: 1,
}

class Trader:
    def __init__(self, params=None):
        if params is None:
            params = PARAMS
        self.params = params

        self.LIMIT = {
            Product.PICNIC_BASKET1: 60,
            Product.CROISSANTS: 250,
            Product.JAMS: 350,
            Product.DJEMBES: 60,
        }

    def get_swmid(self, order_depth: OrderDepth) -> float:
        """Calculate volume-weighted midprice from an order depth."""
        best_bid = max(order_depth.buy_orders.keys())
        best_ask = min(order_depth.sell_orders.keys())
        best_bid_vol = abs(order_depth.buy_orders[best_bid])
        best_ask_vol = abs(order_depth.sell_orders[best_ask])
        return (best_bid * best_ask_vol + best_ask * best_bid_vol) / (best_bid_vol + best_ask_vol)

    def get_synthetic_basket_order_depth(
            self, order_depths: Dict[str, OrderDepth]
    ) -> OrderDepth:
        """
        Construct a synthetic basket order depth from the component order books.

        The synthetic basket is calculated using the weighted prices from:
          - CROISSANTS, JAMS, DJEMBES.
        """
        CROISSANTS_PER_BASKET = BASKET_WEIGHTS[Product.CROISSANTS]
        JAMS_PER_BASKET = BASKET_WEIGHTS[Product.JAMS]
        DJEMBES_PER_BASKET = BASKET_WEIGHTS[Product.DJEMBES]

        synthetic_order_depth = OrderDepth()

        # Retrieve the best bid and ask from each component
        croissants_best_bid = (max(order_depths[Product.CROISSANTS].buy_orders.keys())
                              if order_depths[Product.CROISSANTS].buy_orders else 0)
        croissants_best_ask = (min(order_depths[Product.CROISSANTS].sell_orders.keys())
                              if order_depths[Product.CROISSANTS].sell_orders else float("inf"))
        jams_best_bid = (max(order_depths[Product.JAMS].buy_orders.keys())
                                 if order_depths[Product.JAMS].buy_orders else 0)
        jams_best_ask = (min(order_depths[Product.JAMS].sell_orders.keys())
                                 if order_depths[Product.JAMS].sell_orders else float("inf"))
        djembes_best_bid = (max(order_depths[Product.DJEMBES].buy_orders.keys())
                          if order_depths[Product.DJEMBES].buy_orders else 0)
        djembes_best_ask = (min(order_depths[Product.DJEMBES].sell_orders.keys())
                          if order_depths[Product.DJEMBES].sell_orders else float("inf"))

        # Calculate the synthetic (implied) bid and ask prices
        implied_bid = (croissants_best_bid * CROISSANTS_PER_BASKET +
                       jams_best_bid * JAMS_PER_BASKET +
                       djembes_best_bid * DJEMBES_PER_BASKET)
        implied_ask = (croissants_best_ask * CROISSANTS_PER_BASKET +
                       jams_best_ask * JAMS_PER_BASKET +
                       djembes_best_ask * DJEMBES_PER_BASKET)

        # Calculate the volume available for the synthetic basket orders
        if implied_bid > 0:
            croissants_bid_volume = order_depths[Product.CROISSANTS].buy_orders[croissants_best_bid] // CROISSANTS_PER_BASKET
            jams_bid_volume = order_depths[Product.JAMS].buy_orders[jams_best_bid] // JAMS_PER_BASKET
            djembes_bid_volume = order_depths[Product.DJEMBES].buy_orders[djembes_best_bid] // DJEMBES_PER_BASKET
            implied_bid_volume = min(croissants_bid_volume, jams_bid_volume, djembes_bid_volume)
            synthetic_order_depth.buy_orders[implied_bid] = implied_bid_volume

        if implied_ask < float("inf"):
            croissants_ask_volume = - (order_depths[Product.CROISSANTS].sell_orders[croissants_best_ask] // CROISSANTS_PER_BASKET)
            jams_ask_volume = - (order_depths[Product.JAMS].sell_orders[jams_best_ask] // JAMS_PER_BASKET)
            djembes_ask_volume = - (order_depths[Product.DJEMBES].sell_orders[djembes_best_ask] // DJEMBES_PER_BASKET)
            implied_ask_volume = min(croissants_ask_volume, jams_ask_volume, djembes_ask_volume)
            synthetic_order_depth.sell_orders[implied_ask] = -implied_ask_volume

        return synthetic_order_depth

    def convert_synthetic_basket_orders(
            self, synthetic_orders: List[Order], order_depths: Dict[str, OrderDepth]
    ) -> Dict[str, List[Order]]:
        """
        Converts orders on the synthetic basket (SYNTHETIC) into orders on each constituent market.

        For each synthetic order, if it’s a buy order and the price
        is at or above the best ask of the synthetic basket,
        the code converts it into orders for CROISSANTS, JAMS, and DJEMBES using the corresponding basket weights.
        Similarly for a sell order.
        """
        component_orders = {
            Product.CROISSANTS: [],
            Product.JAMS: [],
            Product.DJEMBES: [],
        }
        synthetic_basket_order_depth = self.get_synthetic_basket_order_depth(order_depths)
        best_bid = (max(synthetic_basket_order_depth.buy_orders.keys())
                    if synthetic_basket_order_depth.buy_orders else 0)
        best_ask = (min(synthetic_basket_order_depth.sell_orders.keys())
                    if synthetic_basket_order_depth.sell_orders else float("inf"))

        for order in synthetic_orders:
            price = order.price
            quantity = order.quantity

            if quantity > 0 and price >= best_ask:
                # For a synthetic buy order, execute component orders at each component's best ask.
                croissants_price = min(order_depths[Product.CROISSANTS].sell_orders.keys())
                jams_price = min(order_depths[Product.JAMS].sell_orders.keys())
                djembes_price = min(order_depths[Product.DJEMBES].sell_orders.keys())
            elif quantity < 0 and price <= best_bid:
                # For a synthetic sell order, execute component orders at each component's best bid.
                croissants_price = max(order_depths[Product.CROISSANTS].buy_orders.keys())
                jams_price = max(order_depths[Product.JAMS].buy_orders.keys())
                djembes_price = max(order_depths[Product.DJEMBES].buy_orders.keys())
            else:
                continue  # Skip orders that don't match the synthetic basket pricing conditions

            # Build orders for each component scaled by the basket weights
            croissants_order = Order(Product.CROISSANTS, croissants_price, quantity * BASKET_WEIGHTS[Product.CROISSANTS])
            jams_order = Order(Product.JAMS, jams_price, quantity * BASKET_WEIGHTS[Product.JAMS])
            djembes_order = Order(Product.DJEMBES, djembes_price, quantity * BASKET_WEIGHTS[Product.DJEMBES])

            component_orders[Product.CROISSANTS].append(croissants_order)
            component_orders[Product.JAMS].append(jams_order)
            component_orders[Product.DJEMBES].append(djembes_order)

        return component_orders

    def execute_spread_orders(
            self,
            target_position: int,
            basket_position: int,
            order_depths: Dict[str, OrderDepth],
    ):
        """
        Executes spread orders between the actual basket (GIFT_BASKET) and the synthetic basket.

        Determines the available volume in the basket order book and in the synthetic basket,
        then creates orders on both sides (real basket and converted synthetic orders) to close the position gap.
        """
        if target_position == basket_position:
            return None

        target_quantity = abs(target_position - basket_position)
        basket_order_depth = order_depths[Product.PICNIC_BASKET1]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(order_depths)

        if target_position > basket_position:
            basket_ask_price = min(basket_order_depth.sell_orders.keys())
            basket_ask_volume = abs(basket_order_depth.sell_orders[basket_ask_price])
            synthetic_bid_price = max(synthetic_order_depth.buy_orders.keys())
            synthetic_bid_volume = abs(synthetic_order_depth.buy_orders[synthetic_bid_price])
            orderbook_volume = min(basket_ask_volume, synthetic_bid_volume)
            execute_volume = min(orderbook_volume, target_quantity)
            basket_orders = [Order(Product.PICNIC_BASKET1, basket_ask_price, execute_volume)]
            synthetic_orders = [Order(Product.SYNTHETIC, synthetic_bid_price, -execute_volume)]
            aggregate_orders = self.convert_synthetic_basket_orders(synthetic_orders, order_depths)
            aggregate_orders[Product.PICNIC_BASKET1] = basket_orders
            return aggregate_orders
        else:
            basket_bid_price = max(basket_order_depth.buy_orders.keys())
            basket_bid_volume = abs(basket_order_depth.buy_orders[basket_bid_price])
            synthetic_ask_price = min(synthetic_order_depth.sell_orders.keys())
            synthetic_ask_volume = abs(synthetic_order_depth.sell_orders[synthetic_ask_price])
            orderbook_volume = min(basket_bid_volume, synthetic_ask_volume)
            execute_volume = min(orderbook_volume, target_quantity)
            basket_orders = [Order(Product.PICNIC_BASKET1, basket_bid_price, -execute_volume)]
            synthetic_orders = [Order(Product.SYNTHETIC, synthetic_ask_price, execute_volume)]
            aggregate_orders = self.convert_synthetic_basket_orders(synthetic_orders, order_depths)
            aggregate_orders[Product.PICNIC_BASKET1] = basket_orders
            return aggregate_orders

    def spread_orders(
            self,
            order_depths: Dict[str, OrderDepth],
            product: Product,
            basket_position: int,
            spread_data: Dict[str, Any],
    ):
        """
        Evaluates if the spread between the midprice of the actual basket and the synthetic basket
        (derived from its constituents) is extreme enough to trigger a trade.

        It tracks a history of spread values to compute a standard deviation and z-score.
        If the z-score exceeds a threshold, the appropriate spread orders are executed.
        """
        if Product.PICNIC_BASKET1 not in order_depths:
            return None

        basket_order_depth = order_depths[Product.PICNIC_BASKET1]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(order_depths)
        basket_swmid = self.get_swmid(basket_order_depth)
        synthetic_swmid = self.get_swmid(synthetic_order_depth)
        spread = basket_swmid - synthetic_swmid
        spread_data.setdefault("spread_history", []).append(spread)

        # Ensure we have enough history for volatility estimation
        window = spread_data.get("spread_std_window", 40)
        if len(spread_data["spread_history"]) < window:
            return None
        elif len(spread_data["spread_history"]) > window:
            spread_data["spread_history"].pop(0)

        spread_std = np.std(spread_data["spread_history"][-window:])
        spread_mean = spread_data.get("default_spread_mean", 43.922489021349016)
        zscore = (spread - spread_mean) / spread_std

        if zscore >= spread_data.get("zscore_threshold", 7):
            if basket_position != -spread_data.get("target_position", 58):
                return self.execute_spread_orders(
                    -spread_data.get("target_position", 58),
                    basket_position,
                    order_depths,
                )
        if zscore <= -spread_data.get("zscore_threshold", 7):
            if basket_position != spread_data.get("target_position", 58):
                return self.execute_spread_orders(
                    spread_data.get("target_position", 58),
                    basket_position,
                    order_depths,
                )

        return None

    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData != None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)

        result = {}
        conversions = 0

        if Product.SPREAD not in traderObject:
            traderObject[Product.SPREAD] = {
                "spread_history": [],
                "prev_zscore": 0,
                "clear_flag": False,
                "curr_avg": 0,
            }

        basket_position = (
            state.position[Product.PICNIC_BASKET1]
            if Product.PICNIC_BASKET1 in state.position
            else 0
        )
        spread_orders = self.spread_orders(
            state.order_depths,
            Product.PICNIC_BASKET1,
            basket_position,
            traderObject[Product.SPREAD],
        )
        if spread_orders != None:
            result[Product.CROISSANTS] = spread_orders[Product.CROISSANTS]
            result[Product.JAMS] = spread_orders[Product.JAMS]
            result[Product.DJEMBES] = spread_orders[Product.DJEMBES]
            result[Product.PICNIC_BASKET1] = spread_orders[Product.PICNIC_BASKET1]

        traderData = jsonpickle.encode(traderObject)

        return result, conversions, traderData

## analyze

In [13]:

import json
import pandas as pd
from collections import defaultdict
import jsonpickle
import numpy as np
import io
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

In [15]:
class Product:
    PICNIC_BASKET1 = "PICNIC_BASKET2"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    SYNTHETIC = "SYNTHETIC" # proxy basket
    SPREAD = "SPREAD" # basket - synthetic

def create_params(sma_window, std_window, zscore_threshold, target_position):
    return {
        Product.SPREAD:{
            "default_spread_mean": 56.922489021349016,
            "default_spread_std": 80.78475819993702,
            "spread_sma_window": sma_window,
            "spread_std_window": std_window,
            "zscore_threshold": zscore_threshold,
            "target_position": target_position
        },
    }

In [18]:
from src.opti.backtester import Backtester
from r3 import Trader
from tqdm import tqdm
import itertools

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

listings = {
    'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
    'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
    'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
    'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),
}

position_limit = {
    'PICNIC_BASKET2': 60,
    'CROISSANTS': 250,
    'JAMS': 350,
    'DJEMBES': 60,
}

fair_calculations = {}

market_data0, trade_history0 = _process_data_('./clean_logs/trade_history_day_-1.log')
market_data1, trade_history1 = _process_data_('./clean_logs/trade_history_day_0.log')
market_data2, trade_history2 = _process_data_('./clean_logs/trade_history_day_1.log')
market_data = market_data0
trade_history = trade_history0

sma_windows = [1500]
std_windows = [40, 50, 60]
zscore_thresholds = [7, 8, 10]
target_positions = [40, 50]

results = []
for sma_window, std_window, zscore_threshold, target_position in tqdm(itertools.product(sma_windows, std_windows, zscore_thresholds, target_positions)):
    total_pnl = 0
    params = create_params(sma_window, std_window, zscore_threshold, target_position)
    trader = Trader(params=params)
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "simple_strat_backtest_no_clear_test_exceed.log")
    backtester.run()
    pnl = sum(float(pnl) for pnl in backtester.pnl.values())
    total_pnl += pnl

    results.append({
        "sma_window": sma_window,
        "std_window": std_window,
        "zscore_threshold": zscore_threshold,
        "target_position": target_position,
        "pnl": total_pnl
    })
    print("="*80)
    print(f"sma_window: {sma_window}, std_window: {std_window}, zscore_threshold: {zscore_threshold}, target_position: {target_position}, pnl: {total_pnl}")
    print("="*80)

df_results = pd.DataFrame(results)
df_results.to_csv("optimization_results.csv", index=False)

1it [00:48, 48.50s/it]

sma_window: 1500, std_window: 40, zscore_threshold: 7, target_position: 40, pnl: 7835.0


2it [01:35, 47.38s/it]

sma_window: 1500, std_window: 40, zscore_threshold: 7, target_position: 50, pnl: 13132.0


3it [02:23, 47.74s/it]

...

sma_window: 1500, std_window: 50, zscore_threshold: 7, target_position: 50, pnl: 20178.0


9it [07:18, 49.22s/it]

sma_window: 1500, std_window: 50, zscore_threshold: 8, target_position: 40, pnl: 16788.0


9it [07:28, 49.78s/it]


ValueError: not enough values to unpack (expected 2, got 0)